In [1]:
import daft

In [2]:
df = daft.read_csv("prompts.csv")
df.show()

actUtf8,promptUtf8,for_devsBoolean
Ethereum Developer,"Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.",true
Linux Terminal,"I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd",true
English Translator and Improver,"I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the improvements and nothing else, do not write explanations. My first sentence is ""istanbulu cok seviyom burada olmak cok guzel""",false
Job Interviewer,"I want you to act as an interviewer. I will be the candidate and you will ask me the interview questions for the `position` position. I want you to only reply as the interviewer. Do not write all the conversation at once. I want you to only do the interview with me. Ask me the questions and wait for my answers. Do not write explanations. Ask me the questions one by one like an interviewer does and wait for my answers. My first sentence is ""Hi""",false
JavaScript Console,"I want you to act as a javascript console. I will type commands and you will reply with what the javascript console should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is console.log(""Hello World"");",true
Excel Sheet,"I want you to act as a text based excel. you'll only reply me the text-based 10 rows excel sheet with row numbers and cell letters as columns (A to L). First column header should be empty to reference row number. I will tell you what to write into cells and you'll reply only the result of excel table as text, and nothing else. Do not write explanations. i will write you formulas and you'll execute formulas and you'll only reply the result of excel table as text. First, reply me the empty sheet.",true
English Pronunciation Helper,"I want you to act as an English pronunciation assistant for Turkish speaking people. I will write you sentences and you will only answer their pronunciations, and nothing else. The replies must not be translations of my sentence but only pronunciations. Pronunciations should use Turkish Latin letters for phonetics. Do not write explanations on replies. My first sentence is ""how the weather is in Istanbul?""",false
Spoken English Teacher and Improver,"I want you to act as a spoken English teacher and improver. I will speak to you in English and you will reply to me in English to practice my spoken English. I want you to keep your reply neat, limiting the reply to 100 words. I want you to strictly correct my grammar mistakes, typos, and factual erro

In [3]:
import json
import os

import requests

api_key = os.getenv("OPENAI_API_KEY")
if api_key is None:
    raise RuntimeError("Please specify your OpenAI API key as the environment variable `OPENAI_API_KEY`.")

headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}


@daft.udf(return_dtype=daft.DataType.string())
def run_gpt4o_on_urls(prompts):
    """Run the gpt-4o LLM by making an API call to OpenAI."""
    results = []
    for prompt in prompts.to_pylist():
        payload = {
            "model": "gpt-4o",
            "messages": [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                    ],
                }
            ],
            "max_tokens": 300,
        }

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
        results.append(json.dumps(response.json()))

    return results

In [4]:
df = df.with_column("gpt_output", run_gpt4o_on_urls(df["prompt"]))
df.show(3)

actUtf8,promptUtf8,for_devsBoolean,gpt_outputUtf8
Ethereum Developer,"Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.",true,"{""id"": ""chatcmpl-B9j6PntC7vQ4JzOqfuVPk5mPnGC5O"", ""object"": ""chat.completion"", ""created"": 1741658025, ""model"": ""gpt-4o-2024-08-06"", ""choices"": [{""index"": 0, ""message"": {""role"": ""assistant"", ""content"": ""Creating a smart contract for a blockchain-based messenger where only the person who deployed the contract can update the message and everyone can read it involves several considerations. The solution below outlines such a contract in Solidity with the necessary functionalities:\n\n1. **Storing and Updating Message:** The contract will have a state variable to store the message and another to keep track of how many times the message has been updated.\n2. **Access Control:** Only the deployer (owner) of the contract can update the message.\n3. **Reading the Message:** Anyone can read the message and view how many times it has been updated.\n4. **Events:** It's a good practice to emit events to log actions such as message updates.\n\nHere's a simple implementation in Solidity:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract Messenger {\n\n // State variable to store the current message\n string private currentMessage;\n // Counter to keep track of how many times message has been updated\n uint256 public updateCount;\n // The address of the contract deployer\n address public owner;\n\n // Event fired when a message is updated\n event MessageUpdated(string newMessage);\n\n // Modifier to restrict access to only the owner\n modifier onlyOwner() {\n require(msg.sender == owner, \""Only the owner can update the message\"");\n _;\n }\n\n // Constructor sets the initial message and marks the deploying address as the owner\n constructor(string memory"", ""refusal"": null}, ""logprobs"": null, ""finish_reason"": ""length""}], ""usage"": {""prompt_tokens"": 106, ""completion_tokens"": 300, ""total_tokens"": 406, ""prompt_tokens_details"": {""cached_tokens"": 0, ""audio_tokens"": 0}, ""completion_tokens_details"": {""reasoning_tokens"": 0, ""audio_tokens"": 0, ""accepted_prediction_tokens"": 0, ""rejected_prediction_tokens"": 0}}, ""service_tier"": ""default"", ""system_fingerprint"": ""fp_f9f4fb6dbf""}"
Linux Terminal,"I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd",true,"{""id"": ""chatcmpl-B9j6WpZPE4MDO3bSiVt5rPuDjGxF4"", ""object"": ""chat.completion"", ""created"": 1741658032, ""model"": ""gpt-4o-2024-08-06"", ""choices"": [{""index"": 0, ""message"": {""role"": ""assistant"", ""content"": ""```bash\n/home/username\n```"", ""refusal"": null}, ""logprobs"": null, ""finish_reason"": ""stop""}], ""usage"": {""prompt_tokens"": 98, ""completion_tokens"": 9, ""total_tokens"": 107, ""prompt_tokens_details"": {""cached_tokens"": 0, ""audio_tokens"": 0}, ""completion_tokens_details"": {""reasoning_tokens"": 0, ""audio_tokens"": 0, ""accepted_prediction_tokens"": 0, ""rejected_prediction_tokens"": 0}}, ""service_tier"": ""default"", ""system_fingerprint"": ""fp_f9f4fb6

In [6]:
df = df.with_column("generated_text", df["gpt_output"].json.query(".choices[0].message.content"))
df.show(3)

actUtf8,promptUtf8,for_devsBoolean,gpt_outputUtf8,generated_textUtf8
Ethereum Developer,"Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.",true,"{""id"": ""chatcmpl-B9j7c9tqKhkr9qPAL12pnfkgIVRa2"", ""object"": ""chat.completion"", ""created"": 1741658100, ""model"": ""gpt-4o-2024-08-06"", ""choices"": [{""index"": 0, ""message"": {""role"": ""assistant"", ""content"": ""Developing a smart contract for a blockchain-based messenger involves creating a Solidity contract that stores public messages on the blockchain. This contract will include features where messages are writable only by the deployer and anyone can read them. It will also track how many times the message has been updated.\n\nBelow is a Solidity implementation for such a smart contract:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract BlockchainMessenger {\n // State variable to store the message\n string public message;\n // State variable to track the number of updates\n uint public updateCount;\n // Address of the contract deployer\n address public owner;\n\n // Modifier to allow only the owner to execute certain functions\n modifier onlyOwner() {\n require(msg.sender == owner, \""Only the contract owner can execute this function\"");\n _;\n }\n\n // Constructor to initialize the message and set the deployer as the owner\n constructor(string memory initialMessage) {\n message = initialMessage;\n owner = msg.sender;\n updateCount = 0;\n }\n\n // Function to update the message, only callable by the owner\n function updateMessage(string memory newMessage) public onlyOwner {\n message = newMessage;\n updateCount++;\n }\n\n // Function to get the current message\n function getMessage() public view returns (string memory){\n return message;\n }\n\n // Function to get the total number of updates\n function"", ""refusal"": null}, ""logprobs"": null, ""finish_reason"": ""length""}], ""usage"": {""prompt_tokens"": 106, ""completion_tokens"": 300, ""total_tokens"": 406, ""prompt_tokens_details"": {""cached_tokens"": 0, ""audio_tokens"": 0}, ""completion_tokens_details"": {""reasoning_tokens"": 0, ""audio_tokens"": 0, ""accepted_prediction_tokens"": 0, ""rejected_prediction_tokens"": 0}}, ""service_tier"": ""default"", ""system_fingerprint"": ""fp_f9f4fb6dbf""}","""Developing a smart contract for a blockchain-based messenger involves creating a Solidity contract that stores public messages on the blockchain. This contract will include features where messages are writable only by the deployer and anyone can read them. It will also track how many times the message has been updated.\n\nBelow is a Solidity implementation for such a smart contract:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract BlockchainMessenger {\n // State variable to store the message\n string public message;\n // State variable to track the number of updates\n uint public updateCount;\n // Address of the contract deployer\n address public owner;\n\n // Modifier to allow only the owner to execute certain functions\n modifier onlyOwner() {\n require(msg.sender == owner, \""Only the contract owner can execute this function\"");\n _;\n }\n\n // Constructor to initialize the message and set the deployer as the owner\n constructor(string memory initialMessage) {\n message = initialMessage;\n owner = msg.sender;\n updateCount = 0;\n }\n\n // Function to update the message, only callable by the owner\n function updateMess

In [7]:
df.write_parquet("generated-text.parquet")

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 ParquetSink: 00:00 

pathUtf8
generated-text.parquet/54bc21a0-fe17-4486-8c74-378b3c080447-0.parquet


In [8]:
daft.read_parquet("generated-text.parquet").show()

actUtf8,promptUtf8,for_devsBoolean,gpt_outputUtf8,generated_textUtf8
Ethereum Developer,"Imagine you are an experienced Ethereum developer tasked with creating a smart contract for a blockchain messenger. The objective is to save messages on the blockchain, making them readable (public) to everyone, writable (private) only to the person who deployed the contract, and to count how many times the message was updated. Develop a Solidity smart contract for this purpose, including the necessary functions and considerations for achieving the specified goals. Please provide the code and any relevant explanations to ensure a clear understanding of the implementation.",true,"{""id"": ""chatcmpl-B9j7rtKq7To9V2leSB9G2fNDtX7AK"", ""object"": ""chat.completion"", ""created"": 1741658115, ""model"": ""gpt-4o-2024-08-06"", ""choices"": [{""index"": 0, ""message"": {""role"": ""assistant"", ""content"": ""To create a smart contract for a blockchain messenger with the specified functionalities, we'll include the following components:\n\n1. **Storage Variables**:\n - A `string` variable to store the message.\n - An `address` variable to store the address of the contract deployer (owner).\n - An `uint256` variable to count updates.\n\n2. **Constructor**:\n - Set the message initially, and store the address of the deployer as the owner.\n\n3. **Functions**:\n - A function to allow the owner to update the message. This function will also increment the update count.\n - A public function to retrieve the current message.\n - A function to retrieve the update count.\n\nHere\u2019s how the Solidity smart contract could be implemented:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract BlockchainMessenger {\n // Storage variables\n string private message;\n address private owner;\n uint256 private updateCount;\n\n // Constructor: initializes the contract with a message and sets the contract creator as the owner\n constructor(string memory initialMessage) {\n message = initialMessage;\n owner = msg.sender;\n updateCount = 0;\n }\n\n // Modifier to restrict function access to the owner\n modifier onlyOwner() {\n require(msg.sender == owner, \""Only the contract owner can call this function.\"");\n _;\n }\n\n // Function to update the message - restricted to the owner\n"", ""refusal"": null}, ""logprobs"": null, ""finish_reason"": ""length""}], ""usage"": {""prompt_tokens"": 106, ""completion_tokens"": 300, ""total_tokens"": 406, ""prompt_tokens_details"": {""cached_tokens"": 0, ""audio_tokens"": 0}, ""completion_tokens_details"": {""reasoning_tokens"": 0, ""audio_tokens"": 0, ""accepted_prediction_tokens"": 0, ""rejected_prediction_tokens"": 0}}, ""service_tier"": ""default"", ""system_fingerprint"": ""fp_f9f4fb6dbf""}","""To create a smart contract for a blockchain messenger with the specified functionalities, we'll include the following components:\n\n1. **Storage Variables**:\n - A `string` variable to store the message.\n - An `address` variable to store the address of the contract deployer (owner).\n - An `uint256` variable to count updates.\n\n2. **Constructor**:\n - Set the message initially, and store the address of the deployer as the owner.\n\n3. **Functions**:\n - A function to allow the owner to update the message. This function will also increment the update count.\n - A public function to retrieve the current message.\n - A function to retrieve the update count.\n\nHere’s how the Solidity smart contract could be implemented:\n\n```solidity\n// SPDX-License-Identifier: MIT\npragma solidity ^0.8.0;\n\ncontract BlockchainMessenger {\n // Storage variables\n string private message;\n address private owner;\n uint256 private updateCount;\n\n // Constructor: initializes the contract with a message and sets the contract creator as the owner\n constructor(string memory initialMessage) {\n message = initialMessage;\n owner = msg.sender;\n updateCount = 0;\n }\n\n // Modifier to restrict function access to the owner\